In [1]:
from nba_api.stats.endpoints import leagueleaders
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import teamyearbyyearstats, playercareerstats, leaguedashteamptshot
import matplotlib
from matplotlib.offsetbox import OffsetImage, AnnotationBbox, DrawingArea, TextArea
from matplotlib.patches import Circle, FancyArrowPatch, FancyArrow, ArrowStyle, Arrow
import matplotlib.ticker as mtick
import time
from tqdm import tqdm
import numpy as np
from nba_api.stats.endpoints import leaguegamelog

In [2]:
box_score = leaguegamelog.LeagueGameLog(direction='DESC',
                                       league_id='00',
                                       season='2023-24',
                                       season_type_all_star='Regular Season',
                                       player_or_team_abbreviation='P').get_data_frames()[0]

In [3]:
box_score

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FANTASY_PTS,VIDEO_AVAILABLE
0,22023,1627734,Domantas Sabonis,1610612758,SAC,Sacramento Kings,0022300901,2024-03-06,SAC @ LAL,W,...,20,12,2,0,3,2,16,11,61.0,1
1,22023,202331,Paul George,1610612746,LAC,LA Clippers,0022300897,2024-03-06,LAC @ HOU,W,...,7,2,1,1,2,0,21,9,36.4,1
2,22023,202695,Kawhi Leonard,1610612746,LAC,LA Clippers,0022300897,2024-03-06,LAC @ HOU,W,...,6,7,2,0,0,1,28,9,51.7,1
3,22023,1627826,Ivica Zubac,1610612746,LAC,LA Clippers,0022300897,2024-03-06,LAC @ HOU,W,...,5,1,0,2,1,1,3,-13,15.5,1
4,22023,1627832,Fred VanVleet,1610612745,HOU,Houston Rockets,0022300897,2024-03-06,HOU vs. LAC,L,...,4,8,2,0,0,2,15,-1,37.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19984,22023,203994,Jusuf Nurkic,1610612756,PHX,Phoenix Suns,0022300062,2023-10-24,PHX @ GSW,W,...,14,3,0,1,1,5,14,2,37.3,1
19985,22023,1626164,Devin Booker,1610612756,PHX,Phoenix Suns,0022300062,2023-10-24,PHX @ GSW,W,...,6,8,1,0,6,4,32,5,48.2,1
19986,22023,1626172,Kevon Looney,1610612744,GSW,Golden State Warriors,0022300062,2023-10-24,GSW vs. PHX,L,...,11,1,0,0,2,2,7,-11,19.7,1
19987,22023,1627780,Gary Payton II,1610612744,GSW,Golden State Warriors,0022300062,2023-10-24,GSW vs. PHX,L,...,2,2,3,0,0,1,1,15,15.4,1


In [4]:
categories = [
    'PTS',
    'REB',
    'AST',
    'BLK',
    'STL',
    'TOV',
    # 'FTM',
    # 'FTA',
    # 'FGM',
    # 'FGA',
    'FG3M',
]

columns_alternative = ['PLAYER_NAME', 'GAME_DATE']

In [5]:
box_score[columns_alternative + categories]

,PLAYER_NAME,GAME_DATE,PTS,REB,AST,BLK,STL,TOV,FG3M
0,Domantas Sabonis,2024-03-06,16,20,12,0,2,3,0
1,Paul George,2024-03-06,21,7,2,1,1,2,4
2,Kawhi Leonard,2024-03-06,28,6,7,0,2,0,1
3,Ivica Zubac,2024-03-06,3,5,1,2,0,1,0
4,Fred VanVleet,2024-03-06,15,4,8,0,2,0,4
...,...,...,...,...,...,...,...,...,...
19984,Jusuf Nurkic,2023-10-24,14,14,3,1,0,1,1
19985,Devin Booker,2023-10-24,32,6,8,0,1,6,3
19986,Kevon Looney,2023-10-24,7,11,1,0,0,2,0
19987,Gary Payton II,2023-10-24,1,2,2,0,3,0,0


In [35]:
box_score.replace("Kevin Porter", "Kevin Porter Jr.", inplace=True)
box_score.replace("Nicolas Claxton", "Nic Claxton", inplace=True)
box_score.replace("Herb Jones", "Herbert Jones", inplace=True)
box_score.replace("C.J. McCollum", "CJ McCollum", inplace=True)
box_score.replace("PJ Washington", "P.J. Washington", inplace=True)
box_score.replace("Gary Trent", "Gary Trent Jr.", inplace=True)

In [45]:
box_score[box_score.PLAYER_NAME == 'P.J. Washington']

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FANTASY_PTS,VIDEO_AVAILABLE
221,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300892,2024-03-05,DAL vs. IND,L,...,7,3,1,1,0,1,20,-10,38.9,1
631,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300871,2024-03-03,DAL vs. PHI,L,...,4,2,1,0,2,2,13,4,21.8,1
787,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300859,2024-03-01,DAL @ BOS,L,...,7,0,0,1,0,2,17,-22,28.4,1
1144,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300844,2024-02-28,DAL @ TOR,W,...,6,0,0,3,2,1,23,-2,37.2,1
1353,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300832,2024-02-27,DAL @ CLE,L,...,2,2,3,1,0,2,11,17,28.4,1
1691,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300819,2024-02-25,DAL @ IND,L,...,6,1,2,0,4,3,2,-17,12.7,1
2218,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300796,2024-02-22,DAL vs. PHX,W,...,6,1,1,1,1,3,12,11,25.7,1
2534,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300785,2024-02-14,DAL vs. SAS,W,...,6,2,0,1,1,2,3,19,15.2,1
3031,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300768,2024-02-12,DAL vs. WAS,W,...,5,3,0,0,2,2,9,8,17.5,1
3260,22023,1629023,P.J. Washington,1610612742,DAL,Dallas Mavericks,0022300748,2024-02-10,DAL vs. OKC,W,...,5,1,1,0,2,3,14,24,22.5,1


In [36]:
top_168 = pd.read_excel('/Users/user/Downloads/BBM_PlayerRankings.xls')

top_168.Name

0      Shai Gilgeous-Alexander
1                  Joel Embiid
2                 Nikola Jokic
3                Anthony Davis
4                  Luka Doncic
                ...           
163               Goga Bitadze
164                Josh Giddey
165              Dyson Daniels
166            Jaden McDaniels
167                  Paul Reed
Name: Name, Length: 168, dtype: object

In [37]:
top_168_names = list(top_168.Name)

In [38]:
table = []

for player in tqdm(top_168_names):
    row = [player]
    for cat in categories:
        cat_row = box_score[box_score.PLAYER_NAME == player][cat]
        mean = np.mean(cat_row)
        std = np.std(cat_row)
        row.extend([mean, std])
        # print(f'{cat} - {player} - {np.round(mean, 2)} +/- {np.round(std, 2)}')
    table.append(row)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [00:00<00:00, 212.23it/s]


In [39]:
df = pd.DataFrame(table)

In [40]:
column_names = []

for cat in categories:
    column_names.extend([f'{cat}', f'{cat}_std'])

In [41]:
df.columns = ['Name'] + column_names

In [44]:
df[df.Name == 'PJ Washington']

,Name,PTS,PTS_std,REB,REB_std,AST,AST_std,BLK,BLK_std,STL,STL_std,TOV,TOV_std,FG3M,FG3M_std
144,PJ Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
new_df = pd.DataFrame.copy(df)

In [18]:
z_scores = {}

for cat in categories:
    mean = np.mean(new_df[cat])
    std = np.std(new_df[cat])
    z_scores[cat] = [mean, std]

In [19]:
for i in tqdm(range(len(new_df))):
    player = new_df.iloc[i] 
    for cat in categories:
        standardized_value = (player[cat] - z_scores[cat][0]) / z_scores[cat][1]
        worst_standardized_value = (player[cat] - player[f'{cat}_std'] - z_scores[cat][0]) / z_scores[cat][1]
        best_standardized_value = (player[cat] + player[f'{cat}_std'] - z_scores[cat][0]) / z_scores[cat][1]
        new_df.at[i, f'{cat}_V'] = standardized_value
        new_df.at[i, f'best_{cat}_V'] = best_standardized_value
        new_df.at[i, f'worst_{cat}_V'] = worst_standardized_value

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [00:00<00:00, 3484.11it/s]


In [20]:
z_scores_names = []
best_z_scores_names = []
worst_z_scores_names = []
for cat in categories:
    z_scores_names.append(f'{cat}_V')
    best_z_scores_names.append(f'best_{cat}_V')
    worst_z_scores_names.append(f'worst_{cat}_V')


In [21]:
new_df['z_score'] = new_df[z_scores_names].mean(axis=1)
new_df['best_z_score'] = new_df[best_z_scores_names].mean(axis=1)
new_df['worst_z_score'] = new_df[worst_z_scores_names].mean(axis=1)

In [22]:
df[df.Name == 'PJ Washington']

,Name,PTS,PTS_std,REB,REB_std,AST,AST_std,BLK,BLK_std,STL,STL_std,TOV,TOV_std,FG3M,FG3M_std
144,PJ Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
